In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from s_dbw import S_Dbw
import math

In [2]:
# resets index after selections have been made
def resetindex(data):
    data=data.reset_index()
    data=data.drop('index', axis=1)
    return data

In [3]:
#reduces the data set to the last iteration of every combination
def lastiteration(data):
    i=0
    j=0
    n=len(data.index)    
    while i<n-1:
        if  (data.iat[j,4]==data.iat[j+1,4] and data.iat[j,5]==data.iat[j+1,5] and data.iat[j,10]<data.iat[j+1,10]):
            data=data.drop(i, axis=0)
            i=i+1
        else:
            i=i+1
            j=j+1
    data=resetindex(data)
    return data

In [251]:
df=pd.read_csv('PCGDim16129.txt', sep='\t')
df=df.drop('Convergence Mode', axis=1)

beta=1e10
dfBeta=df.query(f"Beta=={beta}")
dfBeta=resetindex(dfBeta)

In [252]:
dfBetalast=lastiteration(dfBeta)

In [253]:
dfBetalast['TimeAtSeriesLengthIteration']=''
dfBetalast.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ILUTNonzeros                 90 non-null     int64  
 1   Number of unknowns           90 non-null     int64  
 2   Alpha                        90 non-null     int64  
 3   Beta                         90 non-null     float64
 4   Droptol                      90 non-null     float64
 5   Fillfactor                   90 non-null     int64  
 6   Tolerance                    90 non-null     float64
 7   InitialResidualNorm          90 non-null     float64
 8   ResidualNorm                 90 non-null     float64
 9   ResidualQuotient             90 non-null     float64
 10  Iterations                   90 non-null     int64  
 11  SolutionTime                 90 non-null     float64
 12  ILUAssemblyTime              90 non-null     float64
 13  AbsoluteError         

In [254]:
n=len(dfBetalast.index)
for i in range(0,n):
    dfBetalast.iat[i,15]=dfBetalast.iat[i,11]+dfBetalast.iat[i,12]

In [255]:
ProvenanceDataLast=dfBetalast[['ILUTNonzeros','Droptol','Fillfactor','ResidualNorm', 'Iterations','TimeAtSeriesLengthIteration']]

In [258]:
ProvenanceDataLast
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(resetindex((ProvenanceDataLast.query("ILUTNonzeros<76000")).sort_values(by=['Iterations','TimeAtSeriesLengthIteration'])))

    ILUTNonzeros       Droptol  Fillfactor  ResidualNorm  Iterations  \
0          16129  1.000000e+00           5  3.074430e-17         128   
1          16129  1.000000e+04           5  3.074430e-17         128   
2          16129  1.000000e+02           5  3.074430e-17         128   
3          16129  1.000000e+04           8  3.074430e-17         128   
4          16129  1.000000e+02           0  3.074430e-17         128   
5          16129  1.000000e-04           0  3.074430e-17         128   
6          16129  1.000000e+02           1  3.074430e-17         128   
7          16129  1.000000e+04           3  3.074430e-17         128   
8          16129  1.000000e+00           1  3.074430e-17         128   
9          16129  1.000000e+02           4  3.074430e-17         128   
10         16129  1.000000e+04           0  3.074430e-17         128   
11         16129  1.000000e+00           0  3.074430e-17         128   
12         16129  1.000000e+04           1  3.074430e-17        

In [260]:
dfBetaLimit=dfBeta.query("Iterations<10")
dfBetaLimit=resetindex(dfBetaLimit)

In [261]:
dfBetaLimit

,ILUTNonzeros,Number of unknowns,Alpha,Beta,Droptol,Fillfactor,Tolerance,InitialResidualNorm,ResidualNorm,ResidualQuotient,Iterations,SolutionTime,ILUAssemblyTime,AbsoluteError,RelativeError
0,16129,16129,1,1.000000e+10,1.000000e-12,0,1.000000e-14,7.452470e+12,9.321640e+13,12.50810,1,0.001481,0.012002,0.201276,0.990656
1,16129,16129,1,1.000000e+10,1.000000e-12,0,1.000000e-14,7.452470e+12,8.770630e+13,11.76870,2,0.002185,0.011909,0.199391,0.981377
2,16129,16129,1,1.000000e+10,1.000000e-12,0,1.000000e-14,7.452470e+12,8.261400e+13,11.08540,3,0.002042,0.011889,0.197517,0.972156
3,16129,16129,1,1.000000e+10,1.000000e-12,0,1.000000e-14,7.452470e+12,7.791040e+13,10.45430,4,0.002866,0.012159,0.195653,0.962982
4,16129,16129,1,1.000000e+10,1.000000e-12,0,1.000000e-14,7.452470e+12,7.356900e+13,9.87176,5,0.003594,0.012112,0.193797,0.953847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,16129,16129,1,1.000000e+10,1.000000e+04,9,1.000000e-14,7.452470e+12,7.356900e+13,9.87176,5,0.003405,0.012035,0.193797,0.953847
676,16129,16129,1,1.000000e+10,1.000000e+04,9,1.000000e-14,7.452470e+12,6.956530e+13,9.33453,6,0.003835,0.011799,0.191947,0.944741
677,16129,16129,1,1.000000e+10,1.000000e+04,9,1.000000e-14,7.452470e+12,6.587710e+13,8.83962,7,0.004307,0.011771,0.190101,0.935653
678,16129,16129,1,1.000000e+10,1.000000e+04,9,1.000000e-14,7.452470e+12,6.248380e+13,8.38431,8,0.005616,0.011990,0.188256,0.926572


In [262]:
dfBetaLimitlast=lastiteration(dfBetaLimit)

In [263]:
dfBetaLimitlast['TimeAtSeriesLengthIteration']=''
dfBetaLimitlast.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ILUTNonzeros                 90 non-null     int64  
 1   Number of unknowns           90 non-null     int64  
 2   Alpha                        90 non-null     int64  
 3   Beta                         90 non-null     float64
 4   Droptol                      90 non-null     float64
 5   Fillfactor                   90 non-null     int64  
 6   Tolerance                    90 non-null     float64
 7   InitialResidualNorm          90 non-null     float64
 8   ResidualNorm                 90 non-null     float64
 9   ResidualQuotient             90 non-null     float64
 10  Iterations                   90 non-null     int64  
 11  SolutionTime                 90 non-null     float64
 12  ILUAssemblyTime              90 non-null     float64
 13  AbsoluteError         

In [264]:
n=len(dfBetaLimitlast.index)
for i in range(0,n):
    dfBetaLimitlast.iat[i,15]=dfBetaLimitlast.iat[i,11]+dfBetaLimitlast.iat[i,12]

In [265]:
ProvenanceDataLimitLast=dfBetaLimitlast[['ILUTNonzeros','Droptol','Fillfactor','ResidualNorm', 'Iterations','TimeAtSeriesLengthIteration']]

In [268]:
ProvenanceDataLimitLast['TimeAtSeriesLengthIteration'].sum()/ProvenanceDataLast['TimeAtSeriesLengthIteration'].sum()

0.10382523915465147